# Bedrock LLM Agent Tool Use Framework

## Background

In the rapidly evolving landscape of Large Language Model (LLM) applications, developers often encounter unexpected challenges when integrating these models with custom tools and workflows. A significant issue that has emerged, particularly with Anthropic models, is the lack of server-side schema verification for tool responses. This means that, contrary to what some developers might expect, there's no guarantee that the LLM's responses will strictly adhere to the input tool specifications.

This discrepancy can lead to substantial problems, especially when these potentially non-conforming responses are used as inputs for downstream functions or processes. The issue is further complicated when working with popular LLM orchestration frameworks like LangChain.

LangChain and similar frameworks have introduced features like `bind_tools` to support Pydantic models. Their approach typically involves:

1. Converting Pydantic models to OpenAI-format tool specifications
2. Sending these specifications to the model as input
3. Receiving a JSON response from the model
4. Attempting to parse this response back into a filled-out Pydantic object

However, due to the lack of guaranteed schema conformity in the LLM's output, the final parsing step can fail. Within these frameworks, it's often challenging to implement robust client-side verification and effective retry logic at this critical point of potential failure.

This framework was created to address these specific challenges, offering a more flexible and resilient approach to LLM-tool integration, particularly within the AWS Bedrock ecosystem.

## Overview

This framework provides a robust solution for integrating large language models (LLMs) with custom tools, specifically designed for use with AWS Bedrock. It offers a streamlined approach to handling complex, multi-turn conversations that require external data processing or computation. The framework leverages Pydantic for defining tool interfaces, allowing for flexible output options where Pydantic objects can serve as the final desired output or as validated inputs for downstream tools and APIs.

## Key Features

- **Pydantic-based Tool Definitions**: Leverage the power of Pydantic for type-safe and validated tool interfaces.
- **Automatic ToolSpec Conversion**: Seamlessly converts Pydantic models to Bedrock-compatible toolSpecs.
- **Multi-turn Interaction Support**: Handles complex conversation flows with multiple tool calls.
- **Response Parsing**: Ensures LLM outputs adhere to expected schemas.
- **Sophisticated Error Handling**: Implements retry logic with LLM feedback for improved robustness.
- **Flexible Output Options**: Return Pydantic objects for direct use or further processing, or raw assistant messages.
- **Streaming Support**: Enable real-time interaction where needed.
- **Multi-tool Scenario Handling**: Manage conversations requiring multiple different tools.

## How It Works

1. Define your tools using Pydantic models.
2. The framework converts these models to Bedrock toolSpecs using `pydantic_to_toolspec()`.
3. Engage in multi-turn conversations with the LLM, automatically calling tools as needed.
4. Parse and validate LLM responses to ensure schema compliance.
5. Handle errors gracefully, providing feedback to the LLM for potential self-correction.
6. Process tool outputs, either returning Pydantic objects directly for use as final output or as validated input for downstream processes, or perform additional computations as needed.

## Key Components

- `register_models_from_tools()`: Creates a global registry of all Pydantic models, including nested ones.
- `pydantic_to_toolspec()`: Converts Pydantic models to LLM-compatible tool specifications.
- `generate_text()`: Manages LLM interactions with retry logic.
- `process_tool_use()`: Parses LLM tool responses into Pydantic objects, and passes these as inputs to corresponding tool processor functions, returning the output to the model.

## Usage

```python
# Example usage (simplified)
tool_schemas = [WeatherTool, CityInfoTool]

response_dict = call_model_with_tools(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    # Define an initial input prompt
    user_prompt="What's the weather like in New York?",
    # List the Pydantic models to use
    tool_schemas=tool_schemas,
    # Optional: List of functions to process pydantic outputs
    # If None, the Pydantic object itself will be the tool output
    tool_processors=None,
    use_streaming=True,
    invoke_limit=None,
    max_retries=3,
    log_level="INFO",
)

for tool_output in response_dict["tool_calls"]:
    if isinstance(tool_output, WeatherInfo):
        # Access the properties of the WeatherInfo object directly
        print(f"Temperature: {response.temperature}°C")
```

## Why This Framework?

While similar to other LLM orchestration frameworks like LangChain's `bind_tools`, this solution is tailored specifically for AWS Bedrock. It provides a balance of convenience (through Pydantic usage) and compatibility (with Bedrock's toolSpec requirements), making it an excellent choice for developers working within the AWS ecosystem. The framework's flexibility in handling Pydantic objects as both final outputs and inputs for further processing enhances its utility in complex workflows.

## Advanced Features

- **Nested Schema Handling**: Supports complex tool definitions with nested Pydantic models.
- **Flexible Tool Processing**: Tool functions can return Pydantic model instances for direct use, perform complex operations with external APIs, or serve as validated inputs for downstream processes.
- **Comprehensive Logging**: Detailed logging at each step for debugging and monitoring.

## Getting Started

1. Clone the repository
2. Install dependencies: `pip install -r requirements.txt`
3. Define your Pydantic models for tools
4. Set up your AWS credentials for Bedrock access
5. Use the `call_model_with_tools` function to start interacting with your LLM and tools

In [1]:
#%pip install -U loguru boto3 pydantic rich requests Wikipedia-API

# from rich.traceback import install
# install(show_locals=True)

## TODO
- Add way to send streaming to Streamlit
- Add control for system prompt
- Add `inferenceConfig` to converse API
- Add `guardrailConfig` to converse api
- Investigate `guardContent` in system message


https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse_stream.html

In [2]:
# Price per 1,000 input tokens
global BEDROCK_PRICING
BEDROCK_PRICING = {
    "us-east-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-5-sonnet-20240620-v1:0": {"input": 0.003, "output": 0.015},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
        "anthropic.claude-v2:1": {"input": 0.008, "output": 0.024},
        "anthropic.claude-v2": {"input": 0.008, "output": 0.024},
        "anthropic.claude-instant-v1": {"input": 0.0008, "output": 0.0024},
        "meta.llama2-13b-chat-v1": {"input": 0.00075, "output": 0.001},
        "meta.llama2-70b-chat-v1": {"input": 0.00195, "output": 0.00256},
    },
    "us-west-2": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-opus-20240229-v1:0": {"input": 0.015, "output": 0.075},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
        "anthropic.claude-v2:1": {"input": 0.008, "output": 0.024},
        "anthropic.claude-v2": {"input": 0.008, "output": 0.024},
        "anthropic.claude-instant-v1": {"input": 0.0008, "output": 0.0024},
        "meta.llama2-13b-chat-v1": {"input": 0.00075, "output": 0.001},
        "meta.llama2-70b-chat-v1": {"input": 0.00195, "output": 0.00256},
    },
    "eu-west-2": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "sa-east-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "ca-central-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "ap-south-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "ap-southeast-2": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "eu-west-3": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-3-haiku-20240307-v1:0": {"input": 0.00025, "output": 0.00125},
        "anthropic.claude-3-sonnet-20240229-v1:0": {"input": 0.003, "output": 0.015},
    },
    "eu-central-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-v2:1": {"input": 0.008, "output": 0.024},
        "anthropic.claude-v2": {"input": 0.008, "output": 0.024},
        "anthropic.claude-instant-v1": {"input": 0.0008, "output": 0.0024},
    },
    "ap-northeast-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
        "anthropic.claude-v2:1": {"input": 0.008, "output": 0.024},
        "anthropic.claude-v2": {"input": 0.008, "output": 0.024},
        "anthropic.claude-instant-v1": {"input": 0.0008, "output": 0.0024},
    },
    "ap-east-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "ap-southeast-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "ap-southeast-3": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "eu-north-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "eu-west-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "me-south-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "us-east-2": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    },
    "us-west-1": {
        "ai21.j2-mid-v1": {"input": 0.0125, "output": 0.0125},
        "ai21.j2-ultra-v1": {"input": 0.0188, "output": 0.0188},
        "ai21.jamba-instruct-v1:0": {"input": 0.0005, "output": 0.0007},
        "cohere.command-text-v14": {"input": 0.0015, "output": 0.0020},
        "cohere.command-light-text-v14": {"input": 0.0003, "output": 0.0006},
        "cohere.command-r-plus-v1:0": {"input": 0.0030, "output": 0.0150},
        "cohere.command-r-v1:0": {"input": 0.0005, "output": 0.0015},
        "cohere.embed-english-v3": {"input": 0.0001, "output": None},
        "cohere.embed-multilingual-v3": {"input": 0.0001, "output": None},
    }
}

In [11]:
import os
import boto3
from loguru import logger
import time
import json
import inspect
from functools import wraps
from typing import Any, List, Literal, Type, Generator, Callable
from pydantic import BaseModel, Field
from botocore.config import Config
from botocore.exceptions import ClientError
from rich.console import Console
from rich.logging import RichHandler
from rich.status import Status
from contextlib import contextmanager
from collections import Counter


LogLevel = Literal["TRACE", "DEBUG", "INFO", "SUCCESS", "WARNING", "ERROR", "CRITICAL"]

def pydantic_to_toolspec(model: type[BaseModel]) -> dict[str, Any]:
    """
    Convert a Pydantic model to a toolSpec dictionary, including additional field arguments.

    This function takes a Pydantic model and converts it into a toolSpec format,
    which includes the model's name, description (using the model's docstring if available),
    and a JSON schema of its structure, including additional constraints like ge, min_items, and max_items.

    Args:
        model: A Pydantic model class.

    Returns:
        A dictionary representing the toolSpec for the input model.
    """
    def convert_schema(schema: dict[str, Any], defs: dict[str, Any]) -> dict[str, Any]:
        """
        Recursively convert a JSON schema dictionary, resolving any references
        and including additional field arguments.

        Args:
            schema: The schema dictionary to convert.
            defs: The definitions dictionary for resolving references.

        Returns:
            The converted schema dictionary.
        """
        if "$ref" in schema:
            ref_key = schema["$ref"].split("/")[-1]
            return convert_schema(defs[ref_key], defs)

        result: dict[str, Any] = {}
        for key in ["type", "description"]:
            if key in schema:
                result[key] = schema[key]

        # Include additional constraints
        for constraint in ["minimum", "maximum", "exclusiveMinimum", "exclusiveMaximum", "minItems", "maxItems"]:
            if constraint in schema:
                result[constraint] = schema[constraint]

        if "properties" in schema:
            result["properties"] = {
                k: convert_schema(v, defs) for k, v in schema["properties"].items()
            }

        if "items" in schema:
            result["items"] = convert_schema(schema["items"], defs)

        if "required" in schema:
            result["required"] = schema["required"]

        return result

    schema = model.model_json_schema()
    defs = schema.get("$defs", {})

    # Use the model's docstring if it exists, otherwise fall back to the schema description or a default
    description = model.__doc__.strip() if model.__doc__ else schema.get("description", f"Model for {schema['title']}")

    return {
        "toolSpec": {
            "name": schema["title"],
            "description": description,
            "inputSchema": {
                "json": convert_schema(schema, defs)
            }
        }
    }


def format_tools(models: list[type[BaseModel]]) -> list[dict[str, Any]]:
    """
    Convert multiple Pydantic models to a list of toolSpec dictionaries.

    Args:
        models: A list of Pydantic model classes.

    Returns:
        A list of toolSpec dictionaries, one for each input model.
    """
    formatted_tools = [pydantic_to_toolspec(model) for model in models]
    for n, formatted_tool in enumerate(formatted_tools):
        logger.debug(f"Tool {n + 1}:\n{json.dumps(formatted_tool, indent=4)}")
    return formatted_tools


def register_models_from_tools(tool_schemas: List[Type[BaseModel]]):
    """Registers all Pydantic models from the tools list, including nested models."""
    pydantic_model_registry: dict[str, Type[BaseModel]] = {}

    def register_pydantic_model(model: Type[BaseModel]):
        """Registers a Pydantic model in the global registry."""
        pydantic_model_registry[model.__name__] = model


    def register_nested_models(model: Type[BaseModel], visited: set):
        if model in visited:
            return
        visited.add(model)
        register_pydantic_model(model)
        for field in model.model_fields.values():
            field_type = field.annotation
            if inspect.isclass(field_type) and issubclass(field_type, BaseModel):
                register_nested_models(field_type, visited)
            elif (getattr(field_type, '__origin__', None) is list and
                  inspect.isclass(field_type.__args__[0]) and issubclass(field_type.__args__[0], BaseModel)):
                register_nested_models(field_type.__args__[0], visited)

    visited_models = set()
    for model in tool_schemas:
        register_nested_models(model, visited_models)

    return pydantic_model_registry


def stream_messages(bedrock_client, model_id: str, messages: list[dict[str, Any]], system_prompt: dict, tool_config: dict[str, Any]) -> Generator[str, None, tuple[dict, str, dict[str, Any], Counter]]:
    """Streams messages to a model and processes the response."""
    logger.debug(f"Streaming messages with model {model_id}")

    response = bedrock_client.converse_stream(
        modelId=model_id,
        messages=messages,
        system=[system_prompt],
        toolConfig=tool_config
    )

    stop_reason = ""
    message = {"content": []}
    text = ''
    tool_use = {}
    stream_usage = Counter()

    try:
        for chunk in response['stream']:
            if 'metadata' in chunk:
                stream_usage.update(chunk['metadata']['usage'])
            if 'messageStart' in chunk:
                message['role'] = chunk['messageStart']['role']
            elif 'contentBlockStart' in chunk:
                tool = chunk['contentBlockStart']['start']['toolUse']
                tool_use = {'toolUseId': tool['toolUseId'], 'name': tool['name']}
            elif 'contentBlockDelta' in chunk:
                delta = chunk['contentBlockDelta']['delta']
                if 'toolUse' in delta:
                    tool_use['input'] = tool_use.get('input', '') + delta['toolUse']['input']
                elif 'text' in delta:
                    text += delta['text']
                    yield delta['text']  # Yield the text chunk
            elif 'contentBlockStop' in chunk:
                if 'input' in tool_use:
                    tool_use['input'] = json.loads(tool_use['input'])
                    message['content'].append({'toolUse': tool_use})
                    tool_use = {}
                else:
                    message['content'].append({'text': text})
                    text = ''
            elif 'messageStop' in chunk:
                stop_reason = chunk['messageStop']['stopReason']
    except Exception as e:
        logger.error(f"Error during streaming: {str(e)}")
    finally:
        # Ensure we always return the values, even if an exception occurred
        yield response, stop_reason, message, stream_usage



def parse_model(tool: dict[str, Any]) -> BaseModel:
    """Parses a tool dictionary into the appropriate Pydantic model."""
    global pydantic_model_registry

    if not isinstance(tool, dict):
        raise ValueError("Input tool must be a dictionary.")

    model_name = tool.get('name')
    input_data = tool.get('input', {})

    if model_name is None or model_name not in pydantic_model_registry:
        raise ValueError("Invalid or missing 'name' in input tool.")

    model_class = pydantic_model_registry[model_name]
    return model_class.model_validate(input_data)


def process_tool_use(tool: dict[str, Any], messages: list[dict[str, Any]], tool_processors: dict | None) -> None:
    """Processes the tool use request."""

    def default_process_function(parsed_model: BaseModel) -> dict[str, Any]:
        """Default process function for tools."""
        return parsed_model.model_dump()

    try:
        # Parse the tool input into the appropriate Pydantic model
        tool_call = parse_model(tool)

        # Get the tool function from the tool_functions dictionary, or use the default function
        if tool_processors is None:
            tool_function = default_process_function
        else:
            tool_function = tool_processors.get(tool["name"], default_process_function)

        # Process the parsed Pydantic object and get the tool output
        # This could just be a simple model_dump() call to convert the Pydantic object serializable object,
        # Or it could be a more complex function that uses the Pydantic object in some way.
        tool_output = tool_function(tool_call)

        tool_result = {
            "toolUseId": tool['toolUseId'],
            "content": [{"json": tool_output}]
        }

        tool_result_message = {
            "role": "user",
            "content": [{"toolResult": tool_result}]
        }

        logger.debug(f"Tool result: {json.dumps(tool_result_message, indent=4)}")

        messages.append(tool_result_message)

        return tool_call

    except Exception as err:
        error_message = (
            "Pydantic model validation failed. The LLM failed to return a valid response.\n"
            f"Validation error details:\n{str(err)}\n\n"
            f"Tool response that caused this error:\n{json.dumps(tool['input'], indent=4)}\n\n"
        )
        logger.error(error_message)

        raise  # Re-raise the exception to trigger retry


def generate_text(region: str, model_id: str, messages: list[dict[str, Any]], tool_config: dict[str, Any], tool_processors: dict | None, use_streaming: bool, invoke_limit: int | None = None, max_retries: int = 3) -> Generator[str | dict[str, Any], None, None]:
    """Generates text using the supplied Amazon Bedrock model with built-in retry logic."""

    original_messages = messages.copy()
    original_user_message = original_messages[-1]['content'][0]['text']
    error_message_pairs = []

    for attempt in range(max_retries):
        try:
            yield from _generate_text_core(region, model_id, messages, tool_config, tool_processors, use_streaming, invoke_limit)
            return

        except Exception as e:
            logger.warning(
                f"Attempt {attempt + 1} failed. Exception:\n{str(e)}\n\n"
                "Note: If you see failures and retry attempts often, you should improve the Pydantic model definitions and docstring."
            )
            if attempt == max_retries - 1:
                raise

            # Extract the assistant's last failed response's 'input' parts
            last_assistant_message = messages[-1] if messages[-1]['role'] == 'assistant' else None
            tool_inputs = []

            if last_assistant_message:
                for content in last_assistant_message['content']:
                    if 'toolUse' in content:
                        tool_inputs.append(json.dumps(content['toolUse']['input'], indent=4))

            # Add the current error-message pair to the list, including only the 'input' parts
            error_message_pairs.append((str(e), tool_inputs))

            # Reset messages to original state
            messages = original_messages.copy()

            # Create error_context with all previous error-message pairs
            error_context = "\n\nNote: Previous attempts resulted in errors. Here's a summary:\n"
            for i, (err, inputs) in enumerate(error_message_pairs, 1):
                error_context += f"\nAttempt {i} error:\n{err}\n"
                for input_msg in inputs:
                    error_context += f"The tool call(s) that caused this error was:\n{input_msg}\n"

            error_context += (
                "\n\nPlease reflect on the provided schema, your previous responses, and these error messages. "
                "Now try again, ensuring that your tool use response matches the input schema exactly! "
                "Critically important: You will lose points if you fail to provide a syntactically correct response! "
                "Do not output the same incorrect schema again. "
                "Also, before returning a tool, acknowledge the error in your text response, "
                "explain why you think it occurred, and describe how you plan to address it. "
                "Remember that a missing parameter error may be due to the parameter actually missing, "
                "or it could be due to the parameter being present but not in the expected location. "
                "Finally, you should not include a top-level 'properties' key in tool responses."
            )

            # Update the last user message with cumulative error information
            messages[-1]['content'][0]['text'] = original_user_message + error_context

    raise Exception(f"All {max_retries} attempts failed")


def _generate_text_core(region: str, model_id: str, messages: list[dict[str, Any]], tool_config: dict[str, Any], tool_processors: dict | None, use_streaming: bool, invoke_limit: int | None) -> Generator[str | dict[str, Any], None, None]:
    """Core logic for generating text using the Amazon Bedrock model."""
    # Define custom configuration
    config = Config(
        retries={
            'max_attempts': 10,  # Number of retry attempts
            'mode': 'adaptive'   # Retry mode (standard or adaptive)
        },
        read_timeout=120,  # Read timeout in seconds
        connect_timeout=30  # Connect timeout in seconds
    )

    # Create the bedrock client with the custom configuration
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        config=config,
        region_name=region,
    )

    stop_reason = 'tool_use'
    first_tool_use = True
    tool_calls = []
    invoke_count = 0

    system_prompt = {
        "text": (
                    "You are an intelligent assistant capable of calling functions to gather all necessary information to answer a user's question comprehensively. "
                    "Each response should build upon the previous one, combining all gathered information into a final, standalone answer. "
                    "Ensure the final response fully addresses the user's question, presenting it as if you are answering from scratch, "
                    "without assuming the user remembers any information from previous responses. "
                    "Provide the final response directly, without any introductory or transitional phrases."
                    "Finally, you should not include a top-level 'properties' key in tool responses."
                )
    }

    usage = Counter()

    while stop_reason == 'tool_use':
        current_tool_config = tool_config if first_tool_use else {
            "tools": tool_config["tools"],
            "toolChoice": {"auto": {}}
        }
        first_tool_use = False

        logger.info(f"Generating text with model {model_id}")

        logger.debug("Messages:")
        for message in messages:
            role = message["role"]
            context = message["content"]
            logger.debug(f"{role}:")
            for part in context:
                if 'text' in part:
                    logger.debug(f"text:\n{part['text']}")
                elif 'toolUse' in part:
                    logger.debug(f"toolUse:\n{json.dumps(part['toolUse'], indent=4)}")
                elif 'toolResult' in part:
                    logger.debug(f"toolResult:\n{json.dumps(part['toolResult'], indent=4)}")
                else:
                    raise ValueError(f"Unexpected message part: {part}")

        logger.debug("Sending messages to model...")

        if use_streaming:
            generator = stream_messages(bedrock_client, model_id, messages, system_prompt, current_tool_config)

            for item in generator:
                if isinstance(item, str):
                    yield item  # Yield the text chunk
                else:
                    # This is the final yield with the return values
                    response, stop_reason, message, stream_usage = item

            logger.debug(f"Bedrock response (streamed):\n{response}")
            usage.update(stream_usage)

        else:
            response = bedrock_client.converse(
                modelId=model_id,
                messages=messages,
                system=[system_prompt],
                inferenceConfig={"maxTokens": 4096, "temperature": 0},
                toolConfig=current_tool_config
            )
            logger.debug(f"Bedrock response:\n{json.dumps(response, indent=4)}")

            message = response['output']['message']
            stop_reason = response['stopReason']
            usage.update(response["usage"])

        messages.append(message)

        if stop_reason == 'tool_use':
            for content in message['content']:
                if 'toolUse' in content:
                    tool = content['toolUse']
                    logger.debug(f"Requesting tool {tool['name']}. Request: {tool['toolUseId']}. Inputs: {tool['input']}")
                    tool_call = process_tool_use(tool, messages, tool_processors)
                    tool_calls.append(tool_call)

        invoke_count += 1

        if invoke_limit is not None and invoke_count >= invoke_limit:
            logger.info(f"Reached invoke limit of {invoke_limit}. Stopping generation.")
            break

    if not use_streaming:
        for content in message['content']:
            if 'text' in content:
                yield content['text']

    logger.info(f"Total token usage:\n{json.dumps(usage, indent=2)}")

    global BEDROCK_PRICING
    model_pricing = BEDROCK_PRICING[region][model_id]
    input_tokens = usage["inputTokens"]
    output_tokens = usage["outputTokens"]
    total_price = (input_tokens / 1000) * model_pricing["input"] + (output_tokens / 1000) * model_pricing["output"]
    total_price_str = f"${total_price:.3f}"
    usage["totalPrice"] = total_price_str
    logger.info(f"Total price: {total_price_str}")

    response_dict = {
        "messages": messages,
        "tool_calls": tool_calls,
        "usage": usage
    }

    yield response_dict

@contextmanager
def spinning_logger(console, message: str):
    """
    A context manager that displays an animated spinner with a message while a task is in progress.
    """
    with console.status(message, spinner="dots") as status:
        try:
            yield status
        finally:
            status.stop()


def call_model_with_tools(
    user_prompt: str,
    model_id: str,
    tool_schemas: list,
    tool_processors: dict | None,
    log_level: LogLevel = "INFO",
    use_streaming: bool = False,
    invoke_limit: int | None = None,
    max_retries: bool = 3,
) -> Generator[str | dict[str, Any], None, None]:

    start_time = time.time()

    # Configure loguru with Rich using the passed log_level
    # Create a custom Console with increased width
    console = Console(width=160)  # Adjust the width as needed
    logger.remove()  # Remove any existing handlers
    logger.add(
        RichHandler(console=console, markup=False),
        level=log_level,
        format="<level>{level}</level>: <level>{message}</level>"
    )

    #with spinning_logger(console, "Calling model with tools...") as status:

    try:
        # Create a list of messages with the user's question as the first message
        messages = [{"role": "user", "content": [{"text": user_prompt}]}]

        # Register the Pydantic models from the tools list
        global pydantic_model_registry
        pydantic_model_registry = register_models_from_tools(tool_schemas)

        # Format the tools for the model
        tool_config = {"tools": format_tools(tool_schemas)}

        # Generate text using the model
        response_generator = generate_text(
            region="us-east-1",
            model_id=model_id,
            messages=messages,
            tool_config=tool_config,
            tool_processors=tool_processors,
            use_streaming=use_streaming,
            invoke_limit=invoke_limit,
            max_retries=max_retries,
        )

        # Yield all items from the generator
        yield from response_generator

    except Exception as err:
        logger.exception(f"Error calling model with tools.")
        yield {"error": str(err)}
    finally:
        elapsed_time = time.time() - start_time
        logger.info(f"Elapsed time: {elapsed_time:.3f} seconds")
        logger.debug(f"Finished generating response with model {model_id}.")



def process_generator_output(
    generator: Generator[str | dict[str, Any], None, None],
    stream_callback: Callable[[str], None] = lambda x: print(x, end='', flush=True),
    error_callback: Callable[[str], None] = lambda x: print(f"Error occurred: {x}"),
) -> tuple[str, dict[str, Any] | None]:
    """
    Process the output from the response generator.

    Args:
    generator: The generator yielding strings (for streaming) and dicts (for final response)
    stream_callback: Function to call with each text chunk (default: print to console)
    error_callback: Function to call if an error occurs (default: print to console)

    Returns:
    Tuple containing the complete streamed text and the final response dict (or None if no dict received)
    """
    streamed_text = ""
    response_data = None

    for item in generator:
        if isinstance(item, str):
            streamed_text += item
            stream_callback(item)
        elif isinstance(item, dict):
            if "error" in item:
                error_callback(item["error"])
            else:
                response_data = item

    return streamed_text, response_data


def print_response_data(
    response_data: dict[str, Any],
    print_func: Callable[[str], None] = print,
    json_indent: int = 2
) -> None:
    """
    Print the details of the response data.

    Args:
    response_data: The dictionary containing response data including tool calls and usage statistics.
    print_func: The function to use for printing (default is the built-in print function).
    json_indent: The indentation level for JSON output (default is 2 spaces).

    Returns:
    None
    """
    if not response_data:
        print_func("No response data received.")
        return

    print_func("\n--- Response Data Details ---")

    if "tool_calls" in response_data:
        print_func("\nTool Calls:")
        for tool_call in response_data["tool_calls"]:
            if isinstance(tool_call, BaseModel):
                print_func(f"{tool_call.__class__.__name__}: {tool_call.model_dump_json(indent=json_indent)}")
            else:
                print_func(f"Unknown tool call type: {type(tool_call)}")

    if "usage" in response_data:
        print_func("\nUsage Statistics:")
        print_func(json.dumps(response_data["usage"], indent=json_indent))

    if "messages" in response_data:
        print_func("\nMessages:")
        for message in response_data["messages"]:
            print_func(f"Role: {message['role']}")
            for content in message['content']:
                if 'text' in content:
                    print_func(f"Text: {content['text']}")
                elif 'toolUse' in content:
                    print_func(f"Tool Use: {json.dumps(content['toolUse'], indent=json_indent)}")
                elif 'toolResult' in content:
                    print_func(f"Tool Result: {json.dumps(content['toolResult'], indent=json_indent)}")
            print()
    # Add any other relevant sections of response_data here

### Weather and City Information Example (Mock APIs)

This example demonstrates how the assistant uses two mock API tools:

1. WeatherRequest: Simulates getting current weather for a city and country.
2. CityInfoRequest: Simulates retrieving background information about a location.

The assistant must determine which tools to use based on the user's query and combine the mock data into a coherent response. This example uses predefined mock data to simulate API responses.

The interaction occurs in three distinct generations:

1. First Generation (Weather Tool Call):
   The assistant recognizes the need for weather information. It generates a response containing two components:
   a) Text explaining the need to use a weather tool to answer the query.
   b) A tool call in JSON format for the WeatherRequest tool.
   
   This JSON tool call is then parsed back into a corresponding Pydantic object to validate its structure. If valid, it's used as input for a `tool_processor` function. This function may perform additional operations (like making an API call) or simply return the structured output if that's sufficient to answer the question. The result is then appended as a new user message.

2. Second Generation (City Info Tool Call):
   After receiving the weather data, the assistant recognizes that it needs additional background information about the location. It again generates a response with two components:
   a) Text explaining the need for more information about the city.
   b) A tool call in JSON format for the CityInfoRequest tool.
   
   Similarly, this JSON is parsed into a Pydantic object, validated, and processed by its corresponding `tool_processor` function. The result is appended to the messages list.

3. Third Generation (Final Answer):
   With both weather and city information now available, the assistant generates a final text response that combines all the gathered data into a comprehensive answer to the user's original query. This response does not include a tool call.

This multi-turn process allows the assistant to gather all required information step-by-step before providing a complete answer. Users should observe these three separate generations. The first two generations each include both explanatory text and a tool call in JSON format (which is then parsed and processed), while the final generation is a text-only response.

The parsing of JSON tool calls back into Pydantic objects serves multiple purposes:
1. It validates the structure of the LLM's output, ensuring it conforms to the expected schema.
2. It provides type safety and autocompletion for developers working with the tool inputs.
3. It allows for easy integration with `tool_processor` functions, which can be designed to work with these typed objects.

The `tool_processor` functions offer flexibility in how tool calls are handled. They can perform complex operations like API calls, database queries, or computations. Alternatively, if the structured Pydantic object itself contains all necessary information to answer the query, the `tool_processor` might simply return this object, allowing the LLM to use its structured data in formulating the final response.

This approach combines the power of LLM-generated queries with the safety and utility of strongly-typed data structures, enabling robust and flexible tool use in AI applications.

In [12]:
def main() -> None:
    """ Main function to demonstrate calling a model with tools. """

    # Select the model to use
    #model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
    model_id = "anthropic.claude-3-haiku-20240307-v1:0"

    # Define the user prompt template
    user_template = """Provide weather and city information for {city}, {country}."""

    # Format the user prompt with the specified city and country
    user_prompt = user_template.format(city="New York", country="United States")

    # Define Pydantic models for the tool inputs

    class WeatherRequest(BaseModel):
        city: str = Field(..., description="Name of the city to get weather information for")
        country: str = Field(..., description="Country where the city is located")

    class CityInfoRequest(BaseModel):
        city: str = Field(..., description="Name of the city to get information for")
        country: str = Field(..., description="Country where the city is located")

    # List of Pydantic models to register
    tool_schemas = [WeatherRequest, CityInfoRequest]

    # Define tool functions to process the Pydantic models

    def get_weather_info(request: WeatherRequest) -> dict[str, Any]:
        logger.info(f"Getting weather information for {request.city}, {request.country}...")
        # In a real scenario, this function would make an API call to a weather service
        # For this example, we'll return mock data based on the input
        if request.city.lower() == "new york" and request.country.lower() == "united states":
            return {
                "temperature": "75 degrees Fahrenheit",
                "condition": "Partly cloudy",
                "humidity": 65
            }
        elif request.city.lower() == "london" and request.country.lower() == "united kingdom":
            return {
                "temperature": 15.0,
                "condition": "Rainy",
                "humidity": 80
            }
        else:
            raise ValueError(f"Weather information not available for {request.city}, {request.country}")

    def get_city_info(request: CityInfoRequest) -> dict[str, Any]:
        logger.info(f"Getting city information for {request.city}, {request.country}...")
        # In a real scenario, this function would query a database or make an API call
        # For this example, we'll return mock data based on the input
        if request.city.lower() == "new york" and request.country.lower() == "united states":
            return {
                "population": 8_419_000,
                "country": "United States",
                "timezone": "Eastern Time Zone (ET)"
            }
        elif request.city.lower() == "london" and request.country.lower() == "united kingdom":
            return {
                "population": 8_982_000,
                "country": "United Kingdom",
                "timezone": "British Summer Time (BST)"
            }
        else:
            raise ValueError(f"City information not available for {request.city}, {request.country}")

    tool_processors = {
        "WeatherRequest": get_weather_info,
        "CityInfoRequest": get_city_info,
    }

    response_generator = call_model_with_tools(
        model_id=model_id,
        user_prompt=user_prompt,
        tool_schemas=tool_schemas,
        tool_processors=tool_processors,
        use_streaming=True,
        max_retries=3,
        log_level="INFO",
    )

    output_text, response_data = process_generator_output(response_generator)

    print_response_data(response_data)

main()

[08/07/24 17:38:31] INFO     INFO: Generating text with model anthropic.claude-3-haiku-20240307-v1:0                                           ]8;id=474022;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=67631;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#357\357]8;;\

[08/07/24 17:38:32] INFO     INFO: Getting weather information for New York, United States...                                                   ]8;id=105515;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/3912687774.py\3912687774.py]8;;\:]8;id=43606;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/3912687774.py#30\30]8;;\

                    INFO     INFO: Generating text with model anthropic.claude-3-haiku-20240307-v1:0                                           ]8;id=318522;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=796765;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#357\357]8;;\

[08/07/24 17:38:33] INFO     INFO: Getting city information for New York, United States...                                                      ]8;id=466852;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/3912687774.py\3912687774.py]8;;\:]8;id=558210;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/3912687774.py#49\49]8;;\

                    INFO     INFO: Generating text with model anthropic.claude-3-haiku-20240307-v1:0                                           ]8;id=253680;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=675551;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#357\357]8;;\

New York is a major city located in the United States, in the Eastern Time Zone. It has a population of approximately 8.4 million people. The current weather in New York is 75 degrees Fahrenheit, with 65% humidity and partly cloudy skies.

[08/07/24 17:38:34] INFO     INFO: Total token usage:                                                                                          ]8;id=965352;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=45567;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#424\424]8;;\
                             {                                                                                                                                  
                               "inputTokens": 2064,                                                                                                             
                               "outputTokens": 180,                                                                                                             
                               "totalTokens": 2244                                                                                                              
                             }                                                                                                                                  

                    INFO     INFO: Total price: $0.001                                                                                         ]8;id=742225;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=242353;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#433\433]8;;\

                    INFO     INFO: Elapsed time: 2.730 seconds                                                                                 ]8;id=180169;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py\1784460815.py]8;;\:]8;id=137281;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_25396/1784460815.py#511\511]8;;\


--- Response Data Details ---

Tool Calls:
WeatherRequest: {
  "city": "New York",
  "country": "United States"
}
CityInfoRequest: {
  "city": "New York",
  "country": "United States"
}

Usage Statistics:
{
  "inputTokens": 2064,
  "outputTokens": 180,
  "totalTokens": 2244,
  "totalPrice": "$0.001"
}

Messages:
Role: user
Text: Provide weather and city information for New York, United States.

Role: assistant
Tool Use: {
  "toolUseId": "tooluse_jXwz5EVER1y71FiCYbf-BA",
  "name": "WeatherRequest",
  "input": {
    "city": "New York",
    "country": "United States"
  }
}

Role: user
Tool Result: {
  "toolUseId": "tooluse_jXwz5EVER1y71FiCYbf-BA",
  "content": [
    {
      "json": {
        "temperature": "75 degrees Fahrenheit",
        "condition": "Partly cloudy",
        "humidity": 65
      }
    }
  ]
}

Role: assistant
Tool Use: {
  "toolUseId": "tooluse_9cqgeAQPSGagQnwUQB4d9g",
  "name": "CityInfoRequest",
  "input": {
    "city": "New York",
    "country": "United States"
  }
}

### Weather and City Information Example (Real APIs)

This example shows the assistant using two real API tools:

1. WeatherRequest: Gets actual current weather data for a city and country using a weather API.
2. CityInfoRequest: Retrieves real background information about a location using Wikipedia's API.

The assistant needs to decide which tools to use based on the user's question and integrate real-time data from both APIs into a single, informative response. This example interacts with live data sources.

In [5]:
import requests
import wikipediaapi

def main() -> None:
    """Main function to demonstrate calling a model with weather and location info tools."""

    # Select the model to use
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    # Define the user prompt template
    user_template = """Provide information about the current weather and some background for {location}. 
    """

    # Format the user prompt with an example location
    user_prompt = user_template.format(location="New York")

    class LocationNotFoundError(Exception):
        """Raised when a location isn't found."""
        pass

    def get_current_weather(location: str) -> dict[str, str] | None:
        """Returns the current weather for the requested location using wttr.in."""
        logger.info(f"Getting weather information for {location}...")
        try:
            response = requests.get(f"https://wttr.in/{location}?format=%t+%C&u")
            if response.status_code == 200:
                weather_data = response.text.strip().split()
                return {"temperature": weather_data[0], "condition": " ".join(weather_data[1:])}
            else:
                raise LocationNotFoundError(f"Location {location} not found.")
        except requests.RequestException as e:
            logger.error(f"Error fetching weather data: {e}")
            raise LocationNotFoundError(f"Location {location} not found.")

    def get_location_info(location: str, char_limit: int = 2000) -> dict[str, str] | None:
        """Returns background information for the requested location using Wikipedia."""
        logger.info(f"Getting background information for {location}...")
        headers = {
            'User-Agent': 'LocationInfoBot/1.0 (https://example.org/locationbot/; locationbot@example.org)'
        }

        wiki_wiki = wikipediaapi.Wikipedia(
            language='en',
            extract_format=wikipediaapi.ExtractFormat.WIKI,
            user_agent=headers['User-Agent']
        )
        page = wiki_wiki.page(location)

        if page.exists():
            full_text = page.text[:char_limit]
            return {"full_text": full_text}
        else:
            raise LocationNotFoundError(f"Location {location} not found on Wikipedia.")

    class WeatherRequest(BaseModel):
        """Model for requesting the current weather from a location."""
        location: str = Field(..., description="The name of the location for which you want the current weather. Example locations are New York and London.")

    class LocationInfoRequest(BaseModel):
        """Model for requesting background information about a location."""
        location: str = Field(..., description="The name of the location for which you want the background information. Example locations are New York and London.")
        char_limit: int = Field(2000, description="The maximum number of characters to return from the Wikipedia page. Default is 2000.")


    # List of Pydantic models to register
    tool_schemas = [WeatherRequest, LocationInfoRequest]

    # Define tool functions to process the Pydantic models
    def process_weather_request(request: WeatherRequest) -> dict[str, Any]:
        try:
            weather_data = get_current_weather(request.location)
            return {"weather": weather_data}
        except LocationNotFoundError as e:
            return {"error": str(e)}

    def process_location_info_request(request: LocationInfoRequest) -> dict[str, Any]:
        try:
            location_info = get_location_info(request.location, request.char_limit)
            return {"info": location_info}
        except LocationNotFoundError as e:
            return {"error": str(e)}

    tool_processors = {
        "WeatherRequest": process_weather_request,
        "LocationInfoRequest": process_location_info_request,
    }

    response_dict = call_model_with_tools(
        model_id=model_id,
        user_prompt=user_prompt,
        tool_schemas=tool_schemas,
        tool_processors=tool_processors,
        use_streaming=False,
        max_retries=3,
    )

    for item in response_dict["tool_calls"]:
        if isinstance(item, WeatherRequest):
            print(f"WeatherRequest: {item}")
        elif isinstance(item, LocationInfoRequest):
            print(f"LocationInfoRequest: {item}")
        else:  # If response_type is 'messages'
            print(f"Message: {item}")

if __name__ == "__main__":
    main()

[08/07/24 06:00:40] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=653975;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=995579;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:00:41] INFO     INFO: Getting weather information for New York...                                                                  ]8;id=604841;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/3686933929.py\3686933929.py]8;;\:]8;id=749513;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/3686933929.py#23\23]8;;\

[08/07/24 06:00:42] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=242953;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=429772;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:00:44] INFO     INFO: Getting background information for New York...                                                               ]8;id=774961;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/3686933929.py\3686933929.py]8;;\:]8;id=313167;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/3686933929.py#37\37]8;;\

[08/07/24 06:00:45] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=750473;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=921745;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

The current weather in New York is light rain showers with a temperature of 72°F (22°C). New York, often referred to as New York City, is the most populous city in the United States and is located in the state of New York in the northeastern region. It is a global center for finance, arts, media, entertainment, fashion, research and trade. Some key facts about New York City:

- It consists of five boroughs - Manhattan, Brooklyn, Queens, the Bronx, and Staten Island. Manhattan is the most densely populated and famous borough.

- It has iconic landmarks like the Statue of Liberty, Central Park, Empire State Building, Brooklyn Bridge, and Times Square. 

- It is an important center for industries like finance, technology, real estate, media, and entertainment. Wall Street is located in Lower Manhattan.

- It has a diverse population with people from all over the world, contributing to its vibrant culture, cuisine, and arts scene.

- It hosts many world-class museums like the Metropolitan 

[08/07/24 06:00:54] INFO     INFO: Total token usage:                                                                                           ]8;id=933702;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=998822;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#418\418]8;;\
                             {                                                                                                                                  
                               "inputTokens": 2474,                                                                                                             
                               "outputTokens": 445,                                                                                                             
                               "totalTokens": 2919                                                                                                              
                             }                                                                                                                                  

                    INFO     INFO: Total price: $0.014                                                                                          ]8;id=777813;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=951554;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#427\427]8;;\

                    INFO     INFO: Elapsed time: 13.862 seconds                                                                                 ]8;id=227085;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=192527;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#502\502]8;;\

WeatherRequest: location='New York'
LocationInfoRequest: location='New York' char_limit=2000


### Trip Planning Example

This example involves planning a 3-day trip using three tools:

1. WeatherForecast: Gets weather predictions for the destination.
2. TouristAttractions: Finds popular attractions in the area.
3. FlightSearch: Looks for available flights.

The assistant must recognize that all three tools are needed for a complete trip plan. It has to decide the order to use the tools, gather the necessary information, and then create a coherent plan that includes weather, attractions, and travel details.

In [6]:
from typing import List, Dict, Any
from datetime import datetime, timedelta


def main() -> None:
    """ Main function to demonstrate calling a model with multiple tools for trip planning. """

    # Select the model to use
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    # Define the user prompt template
    user_template = """Plan a 3-day trip to {city}, {country} for next month. 
    Provide information about the weather, top attractions, and available flights from {origin_city}. 
    """

    # Format the user prompt with example values
    user_prompt = user_template.format(
        city="Paris",
        country="France",
        origin_city="New York"
    )

    # Define Pydantic models for the tool inputs

    class WeatherForecast(BaseModel):
        city: str = Field(..., description="Name of the city")
        country: str = Field(..., description="Country where the city is located")
        start_date: str = Field(..., description="Start date of the forecast (YYYY-MM-DD)")
        end_date: str = Field(..., description="End date of the forecast (YYYY-MM-DD)")

    class TouristAttractions(BaseModel):
        city: str = Field(..., description="Name of the city")
        country: str = Field(..., description="Country where the city is located")
        num_attractions: int = Field(5, description="Number of top attractions to return")

    class FlightSearch(BaseModel):
        origin: str = Field(..., description="Origin city")
        destination: str = Field(..., description="Destination city")
        date: str = Field(..., description="Date of travel (YYYY-MM-DD)")

    # List of Pydantic models to register
    tool_schemas = [WeatherForecast, TouristAttractions, FlightSearch]

    # Define tool functions to process the Pydantic models

    def get_weather_forecast(request: WeatherForecast) -> Dict[str, Any]:
        logger.info(f"Getting weather forecast for {request.city}, {request.country}...")
        if request.city.lower() == "paris" and request.country.lower() == "france":
            start_date = datetime.strptime(request.start_date, "%Y-%m-%d")
            forecast = []
            for i in range(3):
                date = start_date + timedelta(days=i)
                forecast.append({
                    "date": date.strftime("%Y-%m-%d"),
                    "temperature": 20 + i,  # Simulated temperature increase
                    "condition": "Partly cloudy" if i % 2 == 0 else "Sunny"
                })
            return {"forecast": forecast}
        else:
            return {"error": f"Weather forecast not available for {request.city}, {request.country}"}

    def get_tourist_attractions(request: TouristAttractions) -> Dict[str, Any]:
        logger.info(f"Getting top attractions for {request.city}, {request.country}...")
        if request.city.lower() == "paris" and request.country.lower() == "france":
            attractions = [
                "Eiffel Tower",
                "Louvre Museum",
                "Notre-Dame Cathedral",
                "Arc de Triomphe",
                "Champs-Élysées"
            ]
            return {"attractions": attractions[:request.num_attractions]}
        else:
            return {"error": f"Tourist attractions not available for {request.city}, {request.country}"}

    def search_flights(request: FlightSearch) -> Dict[str, Any]:
        logger.info(f"Searching flights from {request.origin} to {request.destination}...")
        if request.origin.lower() == "new york" and request.destination.lower() == "paris":
            return {
                "flights": [
                    {"airline": "Air France", "departure": "08:00", "arrival": "21:00", "price": 800},
                    {"airline": "Delta", "departure": "10:30", "arrival": "23:30", "price": 750},
                    {"airline": "United", "departure": "14:00", "arrival": "03:00", "price": 700}
                ]
            }
        else:
            return {"error": f"Flight search not available for {request.origin} to {request.destination}"}

    tool_processors = {
        "WeatherForecast": get_weather_forecast,
        "TouristAttractions": get_tourist_attractions,
        "FlightSearch": search_flights,
    }

    response_dict = call_model_with_tools(
        model_id=model_id,
        user_prompt=user_prompt,
        tool_schemas=tool_schemas,
        tool_processors=tool_processors,
        use_streaming=True,
        max_retries=3,
    )

    print(f"response_dict.keys(): {response_dict.keys()}")

main()

[08/07/24 06:00:54] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=667105;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=490421;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

Okay, let me gather the necessary information to plan a comprehensive 3-day trip to Paris, France for next month.

[08/07/24 06:00:57] INFO     INFO: Getting weather forecast for Paris, France...                                                                ]8;id=192533;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1875777834.py\1875777834.py]8;;\:]8;id=398980;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1875777834.py#47\47]8;;\

                    INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=433920;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=229846;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:00:59] INFO     INFO: Getting top attractions for Paris, France...                                                                 ]8;id=920361;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1875777834.py\1875777834.py]8;;\:]8;id=478583;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1875777834.py#63\63]8;;\

...

For a 3-day trip to Paris, France next month, the weather forecast shows pleasant temperatures around 20-22°C (68-72°F) with partly cloudy to sunny skies. The top 5 attractions to visit are the iconic Eiffel Tower, the world-famous Louvre Museum housing the Mona Lisa, the historic Notre-Dame Cathedral, the impressive Arc de Triomphe, and the chic Champs-Élysées boulevard. For flights from New York, options range from $700-$800 roundtrip on major airlines like United, Delta, and Air France, with both morning and afternoon departures available. With decent weather, must-see attractions, and convenient flight options, you'll be all set to fully experience the beauty and culture of Paris over your 3-day stay.

[08/07/24 06:01:13] INFO     INFO: Total token usage:                                                                                           ]8;id=545513;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=26364;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#418\418]8;;\
                             {                                                                                                                                  
                               "inputTokens": 3934,                                                                                                             
                               "outputTokens": 460,                                                                                                             
                               "totalTokens": 4394                                                                                                              
                             }                                                                                                                                  

                    INFO     INFO: Total price: $0.019                                                                                          ]8;id=793206;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=884836;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#427\427]8;;\

                    INFO     INFO: Elapsed time: 19.391 seconds                                                                                 ]8;id=994104;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=795800;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#502\502]8;;\

response_dict.keys(): dict_keys(['messages', 'tool_calls', 'usage'])


### Workout Program Example

This example demonstrates a different approach to using Pydantic models with the assistant:

1. WorkoutPlan: A complex, nested Pydantic model defining a multi-week workout schedule.

Unlike previous examples where Pydantic models specified input parameters for tools, here the Pydantic model defines the structure of the desired output. The assistant must generate a complete workout plan that conforms to this predefined structure.

This complex nested Pydantic structure serves as a rigorous test of the model's ability to adhere to a provided toolSpec. It's important to note that Anthropic LLMs are not guaranteed to produce output that exactly matches the input schema. As a result, Pydantic validation may sometimes fail. When this happens, it triggers a retry mechanism that adds the Pydantic validation failure reason to the message before attempting again.

Note: The docstring of the WorkoutPlan class and detailed descriptions for each field are critical in this example. These comprehensive descriptions significantly improve the likelihood that the LLM will return a response with the correct syntax. Clear and thorough documentation of the expected structure helps guide the model in generating appropriately formatted output.

Practical consideration: In practice, this example will often fail initially and trigger the retry logic if the number of weeks required in the WorkoutPlan is set to a large number (like over 6). This illustrates the challenges of generating extensive, structured content that consistently meets all specified requirements.

In [7]:
import pandas as pd
from IPython.display import display

def main() -> None:
    """ Main function to demonstrate calling a model with tools. 

    This is where you would configure several things, including:
    - The model ID to use
    - The user prompt
    - The Pydantic models to register
    - Tool functions to process the Pydantic models
    - The response type (tools or messages)
    - Whether to use streaming or not
    """


    # Select the model to use
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    # Define the user prompt template
    user_template = """Create a workout program for the below person:

    Age: {age}
    Weight: {weight}
    Gender: {gender}
    Height: {height}

    I want the program to focus on cardio and weight loss.
    """

    # Format the user prompt with example values
    user_prompt = user_template.format(
        age=60,
        weight="220 lbs",
        gender="male",
        height="6'3"
    )

    # Define Pydantic models to make available to the model

    # NOTE: CURRENTLY THERE IS NO GUARANTEE THAT CLAUDE MODELS WILL ADHERE EXACTLY TO THE PROVIDED TOOL SPEC
    # THIS MAY CAUSE PARSING BACK TO THE PYDANTIC OBJECT TO FAIL
    # This is why Pydantic validation and retry logic is important
    # https://github.com/anthropics/anthropic-sdk-python/issues/619

    class WorkoutDay(BaseModel):
        description: str = Field(..., description="Description of the workout for this day.")
        duration: int = Field(..., ge=0, description="Duration of the workout in minutes. Every day requires a duration, even rest days (use 0).")

    class WorkoutWeek(BaseModel):
        days: list[WorkoutDay] = Field(..., min_items=7, max_items=7, description="List of exactly 7 workout days.")

    class WorkoutPlan(BaseModel):
        """Model for a 90-day workout plan."""
        weeks: list[WorkoutWeek] = Field(..., min_items=13, max_items=13, 
                                         description="List of exactly 13 workout weeks. It's crucial that all 13 weeks are included!")

        @property
        def total_duration(self) -> int:
            """Calculate the total duration of the workout plan in minutes."""
            return sum(day.duration for week in self.weeks for day in week.days)

        @property
        def rest_days(self) -> int:
            """Calculate the number of rest days (days with duration 0) in the plan."""
            return sum(1 for week in self.weeks for day in week.days if day.duration == 0)

        def get_week(self, week_number: int) -> WorkoutWeek:
            """Get a specific week's workout plan."""
            if 1 <= week_number <= 13:
                return self.weeks[week_number - 1]
            raise ValueError("Week number must be between 1 and 13.")

        def get_day(self, day_number: int) -> WorkoutDay:
            """Get a specific day's workout plan."""
            if 1 <= day_number <= 90:
                week_index = (day_number - 1) // 7
                day_index = (day_number - 1) % 7
                return self.weeks[week_index].days[day_index]
            raise ValueError("Day number must be between 1 and 90.")

    # List of Pydantic models to register
    tool_schemas = [WorkoutPlan]

    # Define tool functions to process the Pydantic models

    def process_workout_plan(workout_plan: WorkoutPlan) -> dict[str, Any]:
        logger.info("Processing workout plan...")
        # Convert the Pydantic model to a dictionary (since the model itself is not JSON serializable)
        # Optionally, you could also perform additional processing here
        return workout_plan.model_dump()

    tool_processors = {
        "WorkoutPlan": process_workout_plan,
    }

    response_dict = call_model_with_tools(
        model_id=model_id,
        user_prompt=user_prompt,
        tool_schemas=tool_schemas,
        tool_processors=tool_processors,  # In this case, `None` would use the default processing function (and have the same result)
        use_streaming=True,
        invoke_limit=None,  # Set to `1` to stop after generating a schedule, but before generating an explanation of the schedule
        max_retries=4,
    )

    def display_workout_plan(workout_plan: WorkoutPlan) -> None:
        """
        Convert a WorkoutPlan instance to a pandas DataFrame and display it.

        Args:
        workout_plan (WorkoutPlan): An instance of the WorkoutPlan class

        Returns:
        None: Displays the DataFrame in the notebook
        """
        # Create a list to hold all the data
        data = []

        # Iterate through each week and day
        for week_num, week in enumerate(workout_plan.weeks, start=1):
            for day_num, day in enumerate(week.days, start=1):
                data.append({
                    'Week': week_num,
                    'Day': day_num,
                    'Description': day.description,
                    'Duration (minutes)': day.duration
                })

        # Create a DataFrame
        df = pd.DataFrame(data)

        # Set multi-level index
        df.set_index(['Week', 'Day'], inplace=True)

        # Style the DataFrame
        styled_df = df.style.set_properties(**{
            'background-color': 'lightyellow',
            'border-color': 'black',
            'border-style': 'solid',
            'border-width': '1px',
            'text-align': 'left'
        })

        # Highlight rest days (where duration is 0)
        styled_df = styled_df.applymap(lambda x: 'background-color: lightgreen' if x == 0 else '', subset=['Duration (minutes)'])

        # Display the styled DataFrame
        display(styled_df)


    for item in response_dict["tool_calls"]:
        # If the item is a WorkoutPlan object, we can access its properties
        if isinstance(item, WorkoutPlan):
            print(f"Total duration: {item.total_duration} minutes")
            print(f"Number of rest days: {item.rest_days}")
            print(f"Day 5: {item.get_day(5)}")

            # Display the workout plan using a DataFrame
            display_workout_plan(item)
        # Otherwise if we chose to return the messages directly, we can print them
        else:
            print(item)

main()

[08/07/24 06:01:14] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=240549;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=151224;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:02:03] INFO     INFO: Processing workout plan...                                                                                     ]8;id=244084;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/19670872.py\19670872.py]8;;\:]8;id=282018;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/19670872.py#88\88]8;;\

                    INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=394975;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=678216;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

This 90-day workout plan is designed for a 60-year-old male weighing 220 lbs and standing 6'3" tall, with a focus on cardio and weight loss. The program incorporates a variety of low-impact cardio exercises like brisk walking, swimming/water aerobics, stationary cycling, and low-impact aerobics classes. These activities are complemented by resistance training sessions targeting all major muscle groups, with a gradual progression from lighter weights and higher reps to moderately heavier weights and lower reps. 

Each week consists of 5 workout days and 2 rest days. Workout days combine 45-60 minutes of cardio with 45-60 minutes of resistance training, ensuring a balanced approach to burning calories and building muscle. The resistance workouts incorporate 2-3 sets of 12-15 reps for each exercise, focusing on proper form and gradual overload. Rest days are crucial for recovery and preventing overtraining.

The plan also includes flexibility training through yoga, stretching, or gentle m

[08/07/24 06:02:16] INFO     INFO: Total token usage:                                                                                           ]8;id=771186;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=997542;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#418\418]8;;\
                             {                                                                                                                                  
                               "inputTokens": 5771,                                                                                                             
                               "outputTokens": 340,                                                                                                             
                               "totalTokens": 6111                                                                                                              
                             }                                                                                                                                  

                    INFO     INFO: Total price: $0.022                                                                                          ]8;id=589503;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=591985;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#427\427]8;;\

                    INFO     INFO: Elapsed time: 62.356 seconds                                                                                 ]8;id=443378;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=154986;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#502\502]8;;\

Total duration: 2970 minutes
Number of rest days: 26
Day 5: description='Light resistance training focusing on major muscle groups (legs, back, chest, arms). 3 sets of 15 reps for each exercise.' duration=60


### Multi-Database Financial Health Analysis Example

This example demonstrates an LLM agent analyzing a customer's financial health based on a given ID. The agent must sequentially use two tools: CustomerInfoRequest to query a SQL database for customer data, and TransactionInfoRequest to fetch transaction history from a NoSQL database. Key challenges include constructing correct queries for each database, recognizing the need to use both tools in sequence, and determining when sufficient information has been gathered to provide a comprehensive answer. The agent then analyzes the retrieved data to synthesize a summary of the customer's financial status with recommendations. This process tests the agent's ability to follow a multi-step process, integrate information from multiple sources, and generate insights based on the compiled data.

In [8]:
from typing import Any, Dict, List
from pydantic import BaseModel, Field

def main() -> None:

    # Select the model to use
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    # Define the user prompt template
    user_template = """Analyze the financial health of the customer with ID {customer_id}. 
    Consider their credit score and recent transaction history. 
    Provide a summary of their financial status and any recommendations.
    """

    # Format the user prompt with an example customer ID
    user_prompt = user_template.format(customer_id="12345")


    # Simulated SQL database function (PostgreSQL dialect)
    def query_customer_db(query: str) -> list[dict[str, Any]]:
        """Simulates querying a PostgreSQL database for customer information."""
        mock_data: dict[str, list[dict[str, Any]]] = {
            "SELECT customer_id, name, credit_score FROM customers WHERE customer_id = '12345'": 
                [{"customer_id": "12345", "name": "John Doe", "credit_score": 720}],
            "SELECT customer_id, name, credit_score FROM customers WHERE customer_id = '67890'":
                [{"customer_id": "67890", "name": "Jane Smith", "credit_score": 680}]
        }
        result = mock_data.get(query, [])
        if not result:
            logger.warning(f"No data found for query: {query}")
        return result

    # Simulated NoSQL database function (MongoDB-like API)
    def query_transaction_db(collection: str, query: dict[str, Any]) -> list[dict[str, Any]]:
        """Simulates querying a MongoDB-like database for transaction information."""
        mock_data: dict[str, list[dict[str, Any]]] = {
            "transactions": [
                {"customer_id": "12345", "amount": 1000, "date": "2023-05-01", "type": "deposit"},
                {"customer_id": "12345", "amount": 500, "date": "2023-05-15", "type": "withdrawal"},
                {"customer_id": "67890", "amount": 2000, "date": "2023-05-02", "type": "deposit"},
                {"customer_id": "67890", "amount": 1500, "date": "2023-05-20", "type": "withdrawal"}
            ]
        }
        result = [item for item in mock_data.get(collection, []) if all(item.get(k) == v for k, v in query.items())]
        if not result:
            logger.warning(f"No data found for collection: {collection} with query: {query}")
        return result

    class CustomerInfoRequest(BaseModel):
        """
        Model for requesting customer information from the SQL database.

        This model is used to construct a SQL query to fetch customer data from a PostgreSQL database.
        The query should select the customer_id, name, and credit_score columns from the customers table.
        """
        sql_query: str = Field(
            ...,
            description="SQL query to fetch customer information. Use PostgreSQL dialect. "
                        "The query should select customer_id, name, and credit_score from the customers table. "
                        "Example: SELECT customer_id, name, credit_score FROM customers WHERE customer_id = '12345'"
        )

    class TransactionInfoRequest(BaseModel):
        """
        Model for requesting transaction information from the NoSQL database.

        This model is used to construct a query for a MongoDB-like database to fetch transaction data.
        The collection should be 'transactions' and the query should filter by the customer_id.
        """
        collection: str = Field(
            ...,
            description="Name of the collection to query. Should be 'transactions'."
        )
        query: dict[str, Any] = Field(
            ...,
            description="Query parameters for fetching transaction data. "
                        "Should be a dictionary with 'customer_id' as the key and the customer's ID as the value. "
                        "Example: {'customer_id': '12345'}"
        )

    # List of Pydantic models to register
    tool_schemas = [CustomerInfoRequest, TransactionInfoRequest]

    # Define tool functions to process the Pydantic models
    def process_customer_info_request(request: CustomerInfoRequest) -> Dict[str, Any]:
        logger.info(f"Processing customer info request with SQL query: {request.sql_query}")
        try:
            result = query_customer_db(request.sql_query)
            return {"customer_info": result}
        except Exception as e:
            logger.error(f"Error processing customer info request: {e}")
            return {"error": str(e)}

    def process_transaction_info_request(request: TransactionInfoRequest) -> Dict[str, Any]:
        logger.info(f"Processing transaction info request for collection '{request.collection}' with query: {request.query}")
        try:
            result = query_transaction_db(request.collection, request.query)
            return {"transaction_info": result}
        except Exception as e:
            logger.error(f"Error processing transaction info request: {e}")
            return {"error": str(e)}

    tool_processors = {
        "CustomerInfoRequest": process_customer_info_request,
        "TransactionInfoRequest": process_transaction_info_request,
    }

    response_dict = call_model_with_tools(
        model_id=model_id,
        user_prompt=user_prompt,
        tool_schemas=tool_schemas,
        tool_processors=tool_processors,
        use_streaming=False,
        max_retries=3,
    )

    for item in response_dict["tool_calls"]:
        if isinstance(item, CustomerInfoRequest):
            print(f"CustomerInfoRequest: {item}")
        elif isinstance(item, TransactionInfoRequest):
            print(f"TransactionInfoRequest: {item}")
        else:
            print(item)

if __name__ == "__main__":
    main()

[08/07/24 06:02:16] INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=236401;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=674356;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:02:18] INFO     INFO: Processing customer info request with SQL query: SELECT customer_id, name, credit_score FROM customers WHERE ]8;id=421619;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1974164005.py\1974164005.py]8;;\:]8;id=684781;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1974164005.py#86\86]8;;\
                             customer_id = '12345'                                                                                                              

                    INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=607514;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=205116;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

[08/07/24 06:02:19] INFO     INFO: Processing transaction info request for collection 'transactions' with query: {'customer_id': '12345'}       ]8;id=895365;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1974164005.py\1974164005.py]8;;\:]8;id=248002;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/1974164005.py#95\95]8;;\

                    INFO     INFO: Generating text with model anthropic.claude-3-sonnet-20240229-v1:0                                           ]8;id=286283;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=729586;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#359\359]8;;\

Customer 12345, John Doe, has a credit score of 720 which is considered good. Their recent transaction history shows a $1000 deposit on May 1st and a $500 withdrawal on May 15th. Based on this information, John Doe appears to have a stable financial situation with a positive account balance and a good credit standing. However, to provide a more comprehensive assessment, additional details such as income, expenses, debts, and long-term financial goals would be needed. Overall, the available data suggests John Doe is managing their finances responsibly, but ongoing monitoring and planning is recommended to maintain financial health.


[08/07/24 06:02:23] INFO     INFO: Total token usage:                                                                                           ]8;id=146337;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=962086;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#418\418]8;;\
                             {                                                                                                                                  
                               "inputTokens": 2738,                                                                                                             
                               "outputTokens": 293,                                                                                                             
                               "totalTokens": 3031                                                                                                              
                             }                                                                                                                                  

                    INFO     INFO: Total price: $0.013                                                                                          ]8;id=684088;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=971659;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#427\427]8;;\

                    INFO     INFO: Elapsed time: 7.275 seconds                                                                                  ]8;id=443918;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py\489629671.py]8;;\:]8;id=565897;file:///var/folders/7w/gd5b63cn2zn99gw3n1dg3vvw0000gr/T/ipykernel_53038/489629671.py#502\502]8;;\

CustomerInfoRequest: sql_query="SELECT customer_id, name, credit_score FROM customers WHERE customer_id = '12345'"
TransactionInfoRequest: collection='transactions' query={'customer_id': '12345'}


### Longer Workout Example

In [10]:
import pandas as pd
from IPython.display import display
from typing import List, Any
from pydantic import BaseModel, Field

def main() -> None:
    start_time = time.time()

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    user_template = """Create a workout program for the below person:

    Age: {age}
    Weight: {weight}
    Gender: {gender}
    Height: {height}

    I want the program to focus on cardio and weight loss.

    {continuation_prompt}

    Please provide the next 30 days of the workout plan.
    """

    user_info = {
        "age": 60,
        "weight": "220 lbs",
        "gender": "male",
        "height": "6'3"
    }

    class WorkoutDay(BaseModel):
        description: str = Field(..., description="Description of the workout for this day.")
        duration: int = Field(..., ge=0, description="Duration of the workout in minutes. Every day requires a duration, even rest days (use 0).")

    class WorkoutMonth(BaseModel):
        days: list[WorkoutDay] = Field(..., min_items=30, max_items=30, description="A list (not a string!) of exactly 30 workout days.")
        explanation: str = Field(..., description="An explanation of the workout plan from day 1 to end.")

        @property
        def explanation(self) -> str:
            return self.explanation

    tool_schemas = [WorkoutMonth]

    def process_workout_month(workout_month: WorkoutMonth) -> dict[str, Any]:
        return workout_month.model_dump()

    tool_processors = {
        "WorkoutMonth": process_workout_month,
    }

    full_schedule = []

    def display_workout_month(workout_month: WorkoutMonth, month_number: int) -> None:
        data = []
        for day_num, day in enumerate(workout_month.days, start=1):
            data.append({
                'Day': day_num + month_number * 30,
                'Description': day.description,
                'Duration (minutes)': day.duration
            })

        df = pd.DataFrame(data)
        df.set_index('Day', inplace=True)

        styled_df = df.style.set_properties(**{
            'background-color': 'lightyellow',
            'border-color': 'black',
            'border-style': 'solid',
            'border-width': '1px',
            'text-align': 'left'
        })

        styled_df = styled_df.applymap(lambda x: 'background-color: lightgreen' if x == 0 else '', subset=['Duration (minutes)'])

        display(styled_df)

    for month in range(3):
        if month == 0:
            continuation_prompt = ""
        else:
            previous_days = full_schedule[-30:]
            continuation_prompt = "Continue the workout plan based on the following previous 30 days:\n\n"
            for i, day in enumerate(previous_days, start=1):
                continuation_prompt += f"Day {i + (month-1)*30}: {day.description} ({day.duration} minutes)\n"

        user_prompt = user_template.format(**user_info, continuation_prompt=continuation_prompt)

        response_dict = call_model_with_tools(
            model_id=model_id,
            user_prompt=user_prompt,
            tool_schemas=tool_schemas,
            tool_processors=tool_processors,
            use_streaming=True,
            invoke_limit=1,
            max_retries=4,
            log_level="CRITICAL",
        )

        for item in response_dict["tool_calls"]:
            if isinstance(item, WorkoutMonth):
                full_schedule.extend(item.days)
                display_workout_month(item, month)
                print(item.explanation)

    print(f"Total duration: {sum(day.duration for day in full_schedule)} minutes")
    print(f"Number of rest days: {sum(1 for day in full_schedule if day.duration == 0)}")

    elapsed_time = time.time() - start_time
    print(f"Elapsed time: {elapsed_time:.3f} seconds")

main()

/Users/austinmw/mambaforge/lib/python3.10/site-packages/pydantic/json_schema.py:2099: PydanticJsonSchemaWarning: Default value <property object at 0x11ae6b650> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


The previous error occurred because the input to the "days" parameter was provided as a string, but the schema expects a list of objects. To address this, I will convert the workout days to a JSON list of objects, ensuring each day has the required "description" and "duration" keys.

,Description,Duration (minutes)
Day,,
1,Brisk walking 2 miles,60
2,Rest day,0
3,Stationary cycling 45 minutes,45
4,"Strength training - squats, lunges, pushups",30
5,Rest day,0
6,Swimming laps 30 minutes,30
7,Rest day,0
8,Brisk walking 3 miles,75
9,Yoga or stretching,45


This 30-day workout program focuses on cardio activities like brisk walking, stationary cycling, swimming, and aerobic classes to promote weight loss and improve cardiovascular health for a 60-year-old male weighing 220 lbs. It incorporates lower impact exercises like yoga, pilates, and strength training 2-3 times per week to build muscle and burn additional calories. The program includes proper rest days to allow for recovery and avoids overexertion. With a balanced mix of cardiovascular, strength, and flexibility training, this plan aims to facilitate sustainable weight loss while improving overall fitness.


/Users/austinmw/mambaforge/lib/python3.10/site-packages/pydantic/json_schema.py:2099: PydanticJsonSchemaWarning: Default value <property object at 0x11ae6b650> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


Here is a 30-day workout plan focused on cardio and weight loss for a 60-year-old male, 6'3" and 220 lbs, building on the previous 30 days:

Day 1: Swimming laps 45 minutes
Day 2: Rest day  
Day 3: Stationary cycling 60 minutes
Day 4: Strength training - lower body (squats, lunges, calf raises) 30 minutes  
Day 5: Brisk walking 3 miles
Day 6: Yoga or stretching 45 minutes
Day 7: Rest day
Day 8: Low-impact aerobics class 60 minutes  
Day 9: Strength training - upper body (pushups, rows, overhead press) 30 minutes
Day 10: Elliptical trainer 60 minutes
Day 11: Rest day
Day 12: Water aerobics class 60 minutes
Day 13: Light hiking 2.5 miles  
Day 14: Pilates class 60 minutes
Day 15: Rest day  
Day 16: Brisk walking 4 miles
Day 17: Strength training - full body 45 minutes
Day 18: Stationary cycling 75 minutes
Day 19: Yoga or stretching 30 minutes  
Day 20: Rest day
Day 21: Swimming laps 60 minutes
Day 22: Dance fitness class 60 minutes
Day 23: Strength training - lower body 30 minutes
Day 24

,Description,Duration (minutes)
Day,,
61,Brisk walking 3 miles,75
62,Rest day,0
63,Stationary cycling 60 minutes,60
64,"Strength training - arms, shoulders",45
65,Rest day,0
66,Low-impact aerobics class 1 hour,60
67,Yoga or stretching,45
68,Swimming laps 45 minutes,45
69,Rest day,0


This 30-day workout program for a 60-year-old male focusing on cardio and weight loss includes a variety of activities like brisk walking, cycling, elliptical, swimming, water aerobics, dance fitness, and hiking. It alternates cardio days with strength training for different muscle groups and flexibility through yoga, Pilates and stretching. Rest days are included for recovery. Workout durations range from 30-100 minutes with an emphasis on lower-impact cardio suitable for this age and fitness level. The variety helps prevent plateaus.
Total duration: 2295 minutes
Number of rest days: 19
Elapsed time: 84.089 seconds


In [ ]:
from bedrock_toolkit.cache_services.dynamodb_cache import DynamoDBCachingService

# Initialize the caching service
cache_service = DynamoDBCachingService(
    table_name="my-bedrock-cache",
    embedding_model_id="amazon.titan-embed-text-v2:0",
    embedding_size=256,
    cache_similarity_threshold=0.95,
    cache_time_interval=300,  # 5 minutes
    max_cache_size=1000,
    region="us-east-1"
)

# Example usage
question = "What is the capital of France?"
response = cache_service.get(question)

if response is None:
    # If not in cache, generate the response (e.g., call an LLM)
    response = "Paris"
    cache_service.put(question, response)
    print(f"Generated new response: {response}")
else:
    print(f"Retrieved from cache: {response}")